# Alignment-based Measure Events

In this notebook, we adapt the alignment-based measure with focus on events that was introduced by van der Aalst et al. in 2012  in the paper: "Replaying history on process models for conformance checking and performance analysis" (doi: https://doi.org/10.1002/widm.1045). <br>
The measure is defined as follows:
<br>
For event log $E$, set of unique activities $E'$, process model $M$, $|\text{sim}(e)|$ as the number of times a state $s$ was visited by the log and $|\text{diff}(e)|$ as the number of unique activities when leaving state $s$, generalization is defined as:

$$Generalization(E,M) = 1-\frac{1}{|E|}\sum_{e\,\text{in}\,E'}|e\,\text{in}\,E|\,*\,{pnew}(|\text{diff}(e)|,|\text{sim}(e)|),\; \text{where} $$
$$pnew(w,n)\begin{cases}
    \frac{w(w+1)}{n(n-1)} & \text{if } n\geq w+2, \\
    1 & \text{otherwise}.
\end{cases}$$ 

If generalization converges towards $0$, it is highly likely that a new event occurs in state $s$, and if generalization converges towards $1$, it is highly unlikely.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
from ocpa.objects.log.importer.ocel import factory as ocel_import_factory
from ocpa.algo.discovery.ocpn import algorithm as ocpn_discovery_factory
from src.models.alignment_measure import alignment_measure_events, filter_case_variants
from src.utils import get_happy_path_log, create_flower_model, generate_variant_model
from ocpa.visualization.log.variants import factory as variants_visualization_factory
from ocpa.objects.log.importer.csv import factory as ocel_import_factory_csv
import pickle

C:\Users\Nikla\PycharmProjects\Generalization in Object-Centric Process Mining\venv\lib\site-packages\pm4py\objects\petri\__init__.py:19: UserWarning: please use the pm4py.objects.petri_net package instead.
  warnings.warn('please use the pm4py.objects.petri_net package instead.')
C:\Users\Nikla\PycharmProjects\Generalization in Object-Centric Process Mining\venv\lib\site-packages\pm4py\objects\petri\obj.py:19: UserWarning: pm4py.objects.petri.obj is deprecated. Please use the class pm4py.objects.petri_net.obj instead.
  warnings.warn("pm4py.objects.petri.obj is deprecated. Please use the class pm4py.objects.petri_net.obj instead.")
C:\Users\Nikla\PycharmProjects\Generalization in Object-Centric Process Mining\venv\lib\site-packages\pm4py\objects\petri\exporter\__init__.py:21: UserWarning: please use the pm4py.objects.petri_net.exporter package instead.
  warnings.warn('please use the pm4py.objects.petri_net.exporter package instead.')
C:\Users\Nikla\PycharmProjects\Generalization in O

# O2C Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [3]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [4]:
value = alignment_measure_events(ocel,ocpn)
value

1.0

### Happy Path Petri Net

In [5]:
happy_path__ocel = get_happy_path_log(filename)

In [6]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [7]:
value = alignment_measure_events(ocel,happy_path_ocpn)
value

1.0

### Flower Model Petri Net

In [8]:
ots = ["order","item","delivery"]

In [9]:
flower_ocpn = create_flower_model(filename,ots)

In [10]:
value = alignment_measure_events(ocel,flower_ocpn)
value

1.0

### Variant Model Petri Net

Import the primarly generated variant log for our measure computation, while we generate the variant model with the original log.

In [11]:
filename_variant = "../src/data/csv/order_process_variant_log.csv" 
object_types = ["order","item","delivery"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_variant = ocel_import_factory_csv.apply(file_path=filename_variant, parameters=parameters)

In [12]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ots = ["order","item","delivery"]
ocel = ocel_import_factory.apply(filename)
variant_ocpn = generate_variant_model(ocel,save_path_logs='../src/data/csv/order_variants/order_variant',object_types = ots,save_path_visuals=f"../reports/figures/order_variant_total.svg" )

In [13]:
value = alignment_measure_events(ocel_variant,variant_ocpn)
value

0.9598

# P2P Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [14]:
filename = "../src/data/jsonocel/p2p-normal.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [15]:
value = alignment_measure_events(ocel,ocpn)
value

0.9997

### Happy Path Petri Net

In [16]:
happy_path__ocel = get_happy_path_log(filename)

In [17]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [18]:
value = alignment_measure_events(ocel,happy_path_ocpn)
value

0.9997

### Flower Model Petri Net

In [19]:
ots = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]

In [20]:
flower_ocpn = create_flower_model(filename,ots)

In [21]:
value = alignment_measure_events(ocel,flower_ocpn)
value

0.9998

### Variant Model Petri Net

Import the primarly generated variant log for our measure computation, while we generate the variant model with the original log.

In [22]:
filename_variant = "../src/data/csv/p2p_variant_log.csv" 
object_types = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_variant = ocel_import_factory_csv.apply(file_path=filename_variant, parameters=parameters)

In [23]:
filename = "../src/data/jsonocel/p2p-normal.jsonocel"
ots = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]
ocel = ocel_import_factory.apply(filename)
variant_ocpn = generate_variant_model(ocel,save_path_logs='../src/data/csv/p2p_variants/p2p_variant',object_types = ots ,save_path_visuals=f"../reports/figures/p2p_variant_total.svg" )

In [24]:
value = alignment_measure_events(ocel_variant,variant_ocpn)
value

0.7852

# BPI-Challenge 2017 Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [25]:
filename = "../src/data/jsonocel/BPI2017-Final.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})


KeyboardInterrupt



In [ ]:
value = alignment_measure_events(ocel,ocpn)
value

### Happy Path Petri Net

In [28]:
happy_path__ocel = get_happy_path_log(filename)

In [29]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [30]:
value = alignment_measure_events(ocel,happy_path_ocpn)
value

1.0

### Flower Model Petri Net

In [31]:
ots = ["application","offer"]

In [32]:
flower_ocpn = create_flower_model(filename,ots)

In [33]:
value = alignment_measure_events(ocel,flower_ocpn)
value

1.0

### Variant Model Petri Net

Import the primarly generated variant log for our measure computation, while we generate the variant model with the original log.

In [3]:
filename_variant = "../src/data/csv/bpi2017_variant_log.csv" 
object_types = ["application","offer"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_variant = ocel_import_factory_csv.apply(file_path=filename_variant, parameters=parameters)

In [4]:
with open("../src/data/csv/bpi_variant_ocpn.pickle", "rb") as file:
    variant_ocpn = pickle.load(file)

print(variant_ocpn)

In [5]:
value = alignment_measure_events(ocel_variant,variant_ocpn)
value

Save the transitions: 100%|██████████| 61085/61085 [1:35:33<00:00, 10.65it/s]


0.849

# Measure with each variant appearing only once in log

In a next step, we also want to see if our logs behave in a similar way, as soon as we use a log for the generated models that consists of each variant appearing only once. In a first step, we need to generate the logs for these filtered subsets and save them as csv (is saved as pandas df after filtering). With this csv, we are able to import the filtered log as ocel log again and calculate the results for our alignment measure. We need to be aware that we do not generate new ocpn, but keep the ones already generated with the full log.

# O2C Log

In [25]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [26]:
filter_case_variants(ocel, 'event_variant','event_id', "../src/data/filtered_traces/order_process_filtered.csv")

In [27]:
filename_filtered = "../src/data/filtered_traces/order_process_filtered.csv"
object_types = ["order","item","delivery"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_filtered = ocel_import_factory_csv.apply(file_path=filename_filtered, parameters=parameters)

In [28]:
print("Number of process executions: "+str(len(ocel_filtered.process_executions)))
print("Number of variants: "+str(len(ocel_filtered.variants)))

Number of process executions: 12
Number of variants: 12


### Standard Petri Net

In [29]:
value = alignment_measure_events(ocel_filtered,ocpn)
value

0.9999

### Happy Path Petri Net

In [30]:
happy_path__ocel = get_happy_path_log(filename)

In [31]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [32]:
value = alignment_measure_events(ocel_filtered,happy_path_ocpn)
value

1.0

### Flower Model Petri Net

In [33]:
ots = ["order","item","delivery"]

In [34]:
flower_ocpn = create_flower_model(filename,ots)

In [35]:
value = alignment_measure_events(ocel_filtered,flower_ocpn)
value

1.0

### Variant Model Petri Net

Import the primarly generated variant log for our measure computation, while we generate the variant model with the original log.

In [36]:
filename_variant = "../src/data/filtered_traces/order_process_variant_filtered.csv" 
object_types = ["order","item","delivery"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_variant = ocel_import_factory_csv.apply(file_path=filename_variant, parameters=parameters)

In [37]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ots = ["order","item","delivery"]
ocel = ocel_import_factory.apply(filename)
variant_ocpn = generate_variant_model(ocel,save_path_logs='../src/data/csv/order_variants/order_variant',object_types = ots,save_path_visuals=f"../reports/figures/order_variant_total.svg" )

In [38]:
value = alignment_measure_events(ocel_variant,variant_ocpn)
value

0.9536

# P2P Log

In [39]:
filename = "../src/data/jsonocel/p2p-normal.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [40]:
filter_case_variants (ocel, 'event_variant','event_id', "../src/data/filtered_traces/p2p-normal_filtered.csv")

In [41]:
filename_filtered = "../src/data/filtered_traces/p2p-normal_filtered.csv"
object_types = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_filtered = ocel_import_factory_csv.apply(file_path=filename_filtered, parameters=parameters)

In a further step, we also generrate the variant log of the filtered event log and save it as csv file that we can import again.

In [47]:
print("Number of process executions: "+str(len(ocel_filtered.process_executions)))
print("Number of variants: "+str(len(ocel_filtered.variants)))

Number of process executions: 20
Number of variants: 20


### Standard Petri Net

In [48]:
value = alignment_measure_events(ocel_filtered,ocpn)
value

0.9947

### Happy Path Petri Net

In [49]:
happy_path__ocel = get_happy_path_log(filename)

In [50]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [51]:
value = alignment_measure_events(ocel_filtered,happy_path_ocpn)
value

0.9947

### Flower Model Petri Net

In [52]:
ots = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]

In [53]:
flower_ocpn = create_flower_model(filename,ots)

In [54]:
value = alignment_measure_events(ocel_filtered,flower_ocpn)
value

0.9965

### Variant Model Petri Net

Import the primarly generated variant log for our measure computation, while we generate the variant model with the original log.

In [55]:
filename_variant = "../src/data/filtered_traces/p2p_variant_filtered.csv" 
object_types = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_variant = ocel_import_factory_csv.apply(file_path=filename_variant, parameters=parameters)

In [56]:
filename = "../src/data/jsonocel/p2p-normal.jsonocel"
ots = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]
ocel = ocel_import_factory.apply(filename)
variant_ocpn = generate_variant_model(ocel,save_path_logs='../src/data/csv/p2p_variants/p2p_variant',object_types = ots ,save_path_visuals=f"../reports/figures/p2p_variant_total.svg" )

In [58]:
value = alignment_measure_events(ocel_variant,variant_ocpn)
value

0.48

# BPI-Challenge 2017 Log

In [26]:
filename = "../src/data/jsonocel/BPI2017-Final.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [27]:
filter_case_variants (ocel, 'event_variant','event_id', "../src/data/filtered_traces/BPI2017-Final_filtered.csv")

In [28]:
filename_filtered = "../src/data/filtered_traces/BPI2017-Final_filtered.csv"
object_types = ["application","offer"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_filtered = ocel_import_factory_csv.apply(file_path=filename_filtered, parameters=parameters)

In a further step, we also generrate the variant log of the filtered event log and save it as csv file that we can import again.

In [ ]:
generate_variant_log(ocel_filtered,"../src/data/filtered_traces/bpi2017_variant_filtered.csv" )

In [29]:
print("Number of process executions: "+str(len(ocel_filtered.process_executions)))
print("Number of variants: "+str(len(ocel_filtered.variants)))

Number of process executions: 5071
Number of variants: 5071


### Standard Petri Net

In [30]:
value = alignment_measure_events(ocel_filtered,ocpn)
value

1.0

### Happy Path Petri Net

In [31]:
happy_path__ocel = get_happy_path_log(filename)

In [32]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [33]:
value = alignment_measure_events(ocel_filtered,happy_path_ocpn)
value

1.0

### Flower Model Petri Net

In [36]:
ots = ["application","offer"]

In [37]:
flower_ocpn = create_flower_model(filename,ots)

In [38]:
value = alignment_measure_events(ocel_filtered,flower_ocpn)
value

1.0

### Variant Model Petri Net

In [3]:
filename_filtered = "../src/data/filtered_traces/BPI2017-Final_variant_filtered.csv"
object_types = ["application","offer"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_filtered = ocel_import_factory_csv.apply(file_path=filename_filtered, parameters=parameters)

C:\Users\Nikla\PycharmProjects\Generalization in Object-Centric Process Mining\venv\lib\site-packages\ocpa\objects\log\importer\csv\versions\to_df.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv(filepath, parameters["sep"])


In [4]:
with open("../src/data/csv/bpi_variant_ocpn.pickle", "rb") as file:
    variant_ocpn = pickle.load(file)

print(variant_ocpn)

In [5]:
value = alignment_measure_events(ocel_filtered,variant_ocpn)
value

Save the transitions: 100%|██████████| 61085/61085 [1:26:24<00:00, 11.78it/s]


0.942